# 02. Circuit Screening

This notebook screens the generated circuits based on topological and dynamical properties.

In [1]:
import os
import pandas as pd
import sys

# Add package source to path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

from cytokine_circuits.screening.filters import filter_symmetric, filter_feedback, filter_diverging, filter_N_shaped, filter_fixed_points

In [2]:
input_path = os.path.join(os.getcwd(), '../data/intermediate/df_all_3_interactions.csv')
if not os.path.exists(input_path):
    print("Input file not found. Run 01_generation.ipynb first.")
else:
    df = pd.read_csv(input_path)
    print(f"Initial circuits: {len(df)}")
    
    # 2. Symmetric
    df_symmetric = filter_symmetric(df)
    print(f"After Symmetric filter: {len(df_symmetric)}")
    
    # 3. Feedback
    df_feedback = filter_feedback(df_symmetric)
    print(f"After Feedback filter: {len(df_feedback)}")

    # 4. Diverging
    df_diverging = filter_diverging(df_feedback)
    print(f"After Diverging filter: {len(df_diverging)}")
    
    # 5. Fixed Points
    df_one_fp = filter_fixed_points(df_diverging)
    print(f"After Fixed Point filter: {len(df_one_fp)}")
    
    # 6. Excitability
    df_threshold = filter_N_shaped(df_one_fp)
    print(f"After Excitability filter: {len(df_threshold)}")

Initial circuits: 3584
After Symmetric filter: 1792
After Feedback filter: 640
After Diverging filter: 560
After Fixed Point filter: 544
After Excitability filter: 32


In [4]:
# Model naming and final concatenation as per user's script logic
if 'y_on_x_rem' in df_threshold.columns:
    df_a = df_threshold[(df_threshold['y_on_x_rem']>0) | (df_threshold['y_on_x_prod']<0) ].copy()
    df_b = df_threshold[(df_threshold['y_on_x_rem']<0) | (df_threshold['y_on_x_prod']>0) ].copy()
    df_a['model.num'] = [f'A{i+1}' for i in range(len(df_a))]
    df_b['model.num'] = [f'B{i+1}' for i in range(len(df_a))]
    df_all = pd.concat([df_a,df_b],ignore_index=True)
    
    output_path = os.path.join(os.getcwd(), '../data/intermediate/df_screened_topology.csv')
    df_all.to_csv(output_path, index=False)
    print(f"Saved screened circuits to {output_path}")

Saved screened circuits to /Users/yaelleb/Desktop/Documents/WIS/Alon Lab/Immune Circuit Scan/Excitability_immune_system/Cytokine_circuits_scan/notebooks/../data/intermediate/df_screened_topology.csv
